In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('/content/Data_with_description_9444.csv')

In [3]:
df.head()

,image_link,group_id,entity_name,entity_value,description
0,https://m.media-amazon.com/images/I/51tZoSQjQZ...,101697,width,40.0 centimetre,50 cm 40 cm
1,https://m.media-amazon.com/images/I/612f4sf+uM...,101697,depth,28.0 inch,Thicker than Others Others Ours 0.4mm 1mm Lar...
2,https://m.media-amazon.com/images/I/619S00V9Wp...,101697,depth,20.0 centimetre,"TALLAGRANDE 7.8/20cm 2""/5cm 6.89""/17.5cm"
3,https://m.media-amazon.com/images/I/51iQwQjQ+7...,101697,width,5.0 inch,"Happy Birthday! 4n 5"" 6"" uL 8"" 9"" 10"" 11"" 12"""
4,https://m.media-amazon.com/images/I/514j9sXE6S...,101697,depth,27.2 centimetre,23.3cm/9.17in 27.2cm/10.7in


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11031 entries, 0 to 11030
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_link    11031 non-null  object
 1   group_id      11031 non-null  int64 
 2   entity_name   11031 non-null  object
 3   entity_value  11031 non-null  object
 4   description   9623 non-null   object
dtypes: int64(1), object(4)
memory usage: 431.0+ KB


In [5]:
df.shape

(11031, 5)

In [6]:
df= df.dropna()

In [7]:
df.shape

(9623, 5)

In [8]:
def prepare_input(row):
    return f"group: image_link: {row['image_link']} {row['group_id']} entity_name: {row['entity_name']} description: {row['description']}"

df['input'] = df.apply(prepare_input, axis=1)
df['output'] = df['entity_value']

In [9]:
df.head()

,image_link,group_id,entity_name,entity_value,description,input,output
0,https://m.media-amazon.com/images/I/51tZoSQjQZ...,101697,width,40.0 centimetre,50 cm 40 cm,group: image_link: https://m.media-amazon.com/...,40.0 centimetre
1,https://m.media-amazon.com/images/I/612f4sf+uM...,101697,depth,28.0 inch,Thicker than Others Others Ours 0.4mm 1mm Lar...,group: image_link: https://m.media-amazon.com/...,28.0 inch
2,https://m.media-amazon.com/images/I/619S00V9Wp...,101697,depth,20.0 centimetre,"TALLAGRANDE 7.8/20cm 2""/5cm 6.89""/17.5cm",group: image_link: https://m.media-amazon.com/...,20.0 centimetre
3,https://m.media-amazon.com/images/I/51iQwQjQ+7...,101697,width,5.0 inch,"Happy Birthday! 4n 5"" 6"" uL 8"" 9"" 10"" 11"" 12""",group: image_link: https://m.media-amazon.com/...,5.0 inch
4,https://m.media-amazon.com/images/I/514j9sXE6S...,101697,depth,27.2 centimetre,23.3cm/9.17in 27.2cm/10.7in,group: image_link: https://m.media-amazon.com/...,27.2 centimetre


In [10]:
print(df['input'][0])

group: image_link: https://m.media-amazon.com/images/I/51tZoSQjQZL.jpg 101697 entity_name: width description:  50 cm 40 cm


In [11]:
!pip install datasets

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [13]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [14]:
!pip install torch
!pip install tqdm

In [15]:
!pip install openai-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 2.3 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=7f905a97b14df21e2d3751a23768d01e0ce538c26af9fd200a4e8ca935d39ddc
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip


In [16]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [17]:
train_dataset

Dataset({
    features: ['image_link', 'group_id', 'entity_name', 'entity_value', 'description', 'input', 'output', '__index_level_0__'],
    num_rows: 7698
})

In [19]:
import clip
from PIL import Image
import requests
from io import BytesIO
import torch

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def process_image(image_url):
    # Fetch and open the image
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    # Preprocess the image for CLIP
    image = preprocess(image).unsqueeze(0).to(device)

    return image

def get_clip_features(image_url):
    image = process_image(image_url)
    with torch.no_grad():
        image_features = model.encode_image(image)
    return image_features

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 145MiB/s]


In [25]:
from tqdm import tqdm  # Import tqdm

# Initialize an empty list to store features
features = []

# Wrap df.iterrows() with tqdm to show a progress bar
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Images"):
    image_url = row['image_link']
    image_features = get_clip_features(image_url).cpu().numpy()
    features.append(image_features)

# Convert features to a numpy array for further use
features = np.array(features)

Processing Images: 100%|██████████| 9623/9623 [1:07:57<00:00,  2.36it/s]


In [26]:
print(features)

[[[-0.14570628 -0.08404558  0.15436551 ...  0.2785492   0.20569475
    0.31128418]]

 [[-0.15543391  0.4392404   0.2729646  ...  0.12671673  0.3522673
   -0.23060213]]

 [[-0.7160624   0.05469434  0.18813825 ...  0.56852406 -0.21190077
   -0.12068413]]

 ...

 [[-0.37271404  0.22358708  0.42639798 ... -0.15337807 -0.06647584
   -0.47304824]]

 [[-0.41863915  0.10181977  0.02700306 ...  0.16940811 -0.23574421
    0.23629282]]

 [[-0.65620655  0.11107174 -0.08637391 ...  0.32060224 -0.07590647
    0.01823511]]]


In [29]:
print(f"Shape of features: {features.shape}")

Shape of features: (9623, 1, 512)


In [30]:
if len(features.shape) == 3:
    features = features.squeeze(1)  # Remove the redundant dimension

In [31]:
print(f"Shape of features: {features.shape}")

Shape of features: (9623, 512)


In [32]:
features_list = [feature.tolist() for feature in features]

In [33]:
df['image_features']= features_list

In [34]:
df.head()

,image_link,group_id,entity_name,entity_value,description,input,output,image_features
0,https://m.media-amazon.com/images/I/51tZoSQjQZ...,101697,width,40.0 centimetre,50 cm 40 cm,group: image_link: https://m.media-amazon.com/...,40.0 centimetre,"[-0.14570628106594086, -0.08404558151960373, 0..."
1,https://m.media-amazon.com/images/I/612f4sf+uM...,101697,depth,28.0 inch,Thicker than Others Others Ours 0.4mm 1mm Lar...,group: image_link: https://m.media-amazon.com/...,28.0 inch,"[-0.15543390810489655, 0.43924039602279663, 0...."
2,https://m.media-amazon.com/images/I/619S00V9Wp...,101697,depth,20.0 centimetre,"TALLAGRANDE 7.8/20cm 2""/5cm 6.89""/17.5cm",group: image_link: https://m.media-amazon.com/...,20.0 centimetre,"[-0.7160624265670776, 0.054694343358278275, 0...."
3,https://m.media-amazon.com/images/I/51iQwQjQ+7...,101697,width,5.0 inch,"Happy Birthday! 4n 5"" 6"" uL 8"" 9"" 10"" 11"" 12""",group: image_link: https://m.media-amazon.com/...,5.0 inch,"[-0.3935025632381439, -0.07714860141277313, 0...."
4,https://m.media-amazon.com/images/I/514j9sXE6S...,101697,depth,27.2 centimetre,23.3cm/9.17in 27.2cm/10.7in,group: image_link: https://m.media-amazon.com/...,27.2 centimetre,"[-0.4455071687698364, -0.34295472502708435, 0...."


In [35]:
# Save the DataFrame to a CSV file
df.to_csv('output_with_featres.csv', index=False)

In [36]:
print(df['image_features'][0])

[-0.14570628106594086, -0.08404558151960373, 0.15436550974845886, -0.14421460032463074, 0.40063267946243286, -0.23941577970981598, 0.2860546410083771, 0.47497236728668213, 0.632354736328125, 0.6837124824523926, -0.19286879897117615, -0.1360575407743454, -0.08522646874189377, -0.1394806057214737, 0.23459278047084808, -0.16840220987796783, 0.31842556595802307, 0.36587488651275635, 0.09904134273529053, 0.1980828195810318, 0.2804751694202423, -0.3786559998989105, 0.05116968974471092, -0.26100170612335205, -0.10189618915319443, 0.16502219438552856, -0.02700062096118927, 0.2098412811756134, -0.2375825047492981, 0.05329183116555214, 0.20631255209445953, -0.17937931418418884, -0.3388797342777252, 0.3755093216896057, 0.0004658107936847955, -0.12002139538526535, 0.3415433168411255, -0.45912694931030273, -0.7410603165626526, -0.278531551361084, -0.07050061225891113, -0.07787082344293594, -0.0687410831451416, -0.22259113192558289, 0.1507789045572281, -1.6169594526290894, -0.264449805021286, 0.1276